In [20]:
import numpy as np
import pandas as pd
import awkward
import time
import pickle
import matplotlib
import matplotlib.pyplot as plt

import matplotlib.patches as mpatches

from matplotlib.collections import PatchCollection


In [21]:
from numba import vectorize, int64, float64

@vectorize([int64(int64),float64(float64)])
def getBits(nTC):
    header   = 4 + 5 + 1 ### header(4) + bxID(5) + addressMode(1)                                                                                           
    towerSum = 8
    if nTC < 7:
        total = header + towerSum + 3 + 13*nTC
    else:
        total = header + towerSum + 48 + 7*nTC

    return total


In [22]:
colorConvert = {12:'black',11:'black',10:'black',9:'black',8:'black',7:'black',6:'black',5:'g',4:'b',3:'purple',2:'pink',1:'red'}

import matplotlib.patches as mpatches
patch6 = mpatches.Patch(color='black', label='# eLink > 5')
patch5 = mpatches.Patch(color='green', label='# eLink = 5')
patch4 = mpatches.Patch(color='blue', label='# eLink = 4')
patch3 = mpatches.Patch(color='purple', label='# eLink = 3')
patch2 = mpatches.Patch(color='pink', label='# eLink = 2')
patch1 = mpatches.Patch(color='red', label='# eLink = 1')

legendList=[patch6,patch5,patch4,patch3,patch2,patch1]


In [25]:
threshold=1.5
layer = 36
with open('../mbOccupancy.pkl','rb') as _file:
    mbOccupancy = pickle.load(_file,encoding='bytes')
    
with open('../waferOccupancy_layer%i_thr1p5.pkl'%layer,'rb') as _file:
    waferList = pickle.load(_file,encoding='bytes')
    waferOccup = pickle.load(_file,encoding='bytes')
    

occDF = pd.read_csv("../../HGCAL_Occupancies_JB_v9imp2.csv")
occDF = occDF[(occDF.threshold==1.5) & (occDF.layer==layer)]
occDF.wafer += 2

occDF.loc[occDF.wafer==3,'wafer']=1
occDF.loc[occDF.wafer==6,'wafer']=3
occDF.loc[occDF.wafer==10,'wafer']=6
occDF.loc[occDF.wafer==15,'wafer']=10
occDF.loc[occDF.wafer==21,'wafer']=15
occDF.loc[occDF.wafer==28,'wafer']=21
occDF.loc[occDF.wafer==36,'wafer']=28
occDF.loc[occDF.wafer==45,'wafer']=36
occDF.loc[occDF.wafer==55,'wafer']=45
occDF.loc[occDF.wafer==66,'wafer']=55

motherBoardMap = {
    5: { 1 : 1,    3 : 1,    6 : 1,    2 : 2,    5 : 2,    9 : 2,    4 : 3,    8 : 3,    13: 3,    7 : 4,    11: 4,    16: 4,    10: 5,    15: 5,    21: 5,    12: 6,    18: 6,    25: 6,    14: 7,    20: 7,    27: 7,    17: 8,    24: 8,    32: 8,    19: 9,    26: 9,    34: 9,    22: 10,    29: 10,    37: 10,    23: 11,    31: 11,    40: 11,    28: 12,    36: 12,    30: 13,    38: 13,    47: 13,    33: 14,    41: 14,    42: 14,    35: 15,    43: 15,    44: 15,    39: 16,    48: 16,    49: 16,    50: 17,    60: 17,    61: 17,    51: 18,    52: 18,    53: 18},
    36: { 3  : 1,    6  : 1,    10 : 1,    4  : 2,    7  : 2,    11 : 2,    5  : 3,    9  : 3,    14 : 3,    8  : 4,    12 : 4,    17 : 4,    13 : 5,    19 : 5,    26 : 5,    15 : 6,    21 : 6,    28 : 6,    16 : 7,    22 : 7,    29 : 7,    18 : 8,    25 : 8,    33 : 8,    20 : 9,    27 : 9,    35 : 9,    23 : 10,    30 : 10,    38 : 10,    24 : 11,    31 : 11,    39 : 11,    32 : 12,    41 : 12,    51 : 12,    34 : 13,    43 : 13,    53 : 13,    36 : 14,    45 : 14,    55 : 14,    37 : 15,    46 : 15,    56 : 15,    40 : 16,    50 : 16,    61 : 16,    42 : 17,    52 : 17,    63 : 17,    44 : 18,    54 : 18,    65 : 18,    47 : 19,    57 : 19,    68 : 19,    48 : 20,    58 : 20,    69 : 20,    49 : 21,    59 : 21,    70 : 21,    60 : 22,    71 : 22,    72 : 22,    62 : 23,    73 : 23,    74 : 23,    64 : 24,    75 : 24,    76 : 24,  85:22},
}


occDF.motherboard = occDF.wafer.map(motherBoardMap[layer])

occDF.set_index('wafer',inplace=True)
occDF.sort_index(inplace=True)
occDF.mean_occupancy = np.array([w.mean() for w in waferOccup])

t = occDF.waferv==0
occDF.loc[t,'waferv'] = occDF.loc[t,'waferu']*-1
occDF.loc[t,'waferu'] = 0

occDF['y'] = occDF.waferv
occDF['x'] = 0

occDF.loc[:,'x'] = (-4+occDF.y-2*occDF.waferu)

occDF.y *= 1.5
occDF.x *= 3**.5/2


In [26]:
r = []
nLinks = []
for i,w in enumerate(waferList):
    if w==85: continue
    r.append(mpatches.RegularPolygon((occDF.loc[w].x,occDF.loc[w].y),6,.95))

collection = PatchCollection(r,cmap=matplotlib.cm.nipy_spectral)

with open('layer%i_HexDraw.pkl'%layer,'wb') as _file:
    pickle.dump(collection,_file)
    pickle.dump(occDF,_file)


In [4]:

# nJobs = 1
# thrString = ("%.1f"%threshold).replace('.','p')

# fill = 0
# again = 0

# outputName = 'maxLatency_LinksAboveMin_TTbar_layer%i.pdf'%layer
# if fill>0:
#     outputName = outputName.replace('.pdf','_filled%i.pdf'%fill)
#     if again>0:
#         outputName = outputName.replace('.pdf','_againEvery%i.pdf'%again)

#         tcCount = []
# bitCount = []
# latencyTracker = []
# OverflowCount = []
# ResetCount = []

# for i in range(nJobs):
#     BufferOutput = "/uscmst1b_scratch/lpc1/3DayLifetime/dnoonan/data/ECON-T_Buffer/Output_Buffer_TTbar200PU_thr%s_layer%i_job%i.pkl"%(thrString,layer,i)
#     if fill>0:
#         BufferOutput = BufferOutput.replace('_job','_fillEvery%i_job'%fill)
#         if again>0:
#             BufferOutput = BufferOutput.replace('_job','_againEvery%i_job'%again)
            
            
#     with open(BufferOutput,'rb') as f:
# #         tcCount.append(pickle.load(f))
# #         bitCount.append(pickle.load(f))
# #         latencyTracker.append(pickle.load(f))
# #         OverflowCount.append(pickle.load(f))
# #         ResetCount.append(pickle.load(f))
# #         wafers = pickle.load(f)
# #         meanBits = pickle.load(f)
        
#         tcCount = pickle.load(f)
#         bitCount = pickle.load(f)
#         latencyTracker.append(pickle.load(f))
#         OverflowCount = pickle.load(f)
#         ResetCount = pickle.load(f)
#         wafers = pickle.load(f)
#         meanOcc = pickle.load(f)
#         meanBits = pickle.load(f)
    
    
# keys = OverflowCount.keys()
# N = len(latencyTracker[0][1][0])

# keys = np.array(list(keys))

In [1]:

# cut = meanBits>0

# WaferList = wafers[cut]
# maxLatency = np.array([[latencyTracker[j][k].max() for k in keys] for j in range(nJobs)])
# maxLatency = awkward.fromiter(maxLatency.transpose()).max()[cut]
# _meanBits = meanBits[cut]

# r = []
# nLinks = []
# for i,w in enumerate(WaferList):
#     s = np.round(1.3*_meanBits[i]/32 + .5)
#     nLinks.append(s)
#     r.append(mpatches.RegularPolygon((occDF.loc[w].x,occDF.loc[w].y),6,.95))

# collection = PatchCollection(r,cmap=matplotlib.cm.nipy_spectral)
# collection.set_array(np.array(nLinks))
# collection.set_clim(0,12)
    
    
# fig, ax = plt.subplots()
# colors = np.linspace(0, 1, len(r))

# ax.add_collection(collection)
# if layer==5: plt.axis([-3,15,0,15])
# if layer==36: plt.axis([-3,17,0,18])
# ax.set_xticklabels([])
# ax.set_yticklabels([]);

# cbar = plt.colorbar(collection)
# cbar.set_label('# of eLinks')
